In [ ]:
setwd('C:/Users/Falco/Desktop/directory/Predicting-wildlife-reservoirs-of-Flaviviruses')

In [ ]:
options(warn=-1)
#install.packages(c('gbm', 'dplyr','dismo', 'caret',
#                  'sjplot', 'skmisc','ggplot2'), repos='http://cran.us.r-project.org')
library(gbm)
library(dplyr)
library(dismo)
library(caret)
library(sjPlot)
library(sjmisc)
library(ggplot2)
library(raster)
library(plyr)
library(data.table)

In [ ]:
#model.data = read.csv("Outputs/model_data.csv")
model.data = read.csv('data/dataGroup4.csv')

In [ ]:
#names(model.data)

In [ ]:
model.data$Class = factor(model.data$Class, levels = levels(model.data$Class))
model.data$Order_y = factor(model.data$Order, levels = levels(model.data$Order))
model.data$Family_y = factor(model.data$Family, levels = levels(model.data$Family))
model.data$MatchHost = factor(model.data$MatchHost, levels = levels(model.data$MatchHost))
model.data$IUCN.Status = factor(model.data$IUCN.Status, levels = levels(model.data$IUCN.Status))
model.data$Diet_Certainty = factor(model.data$Diet_Certainty, levels = levels(model.data$Diet_Certainty))
model.data$Group4Binary2 <- ifelse(model.data$Group4Binary==1,'host','nonhost')
model.data$Group4Binary2 <- as.factor(model.data$Group4Binary2)
outcomeName <- 'Group4Binary2'

In [ ]:
## Read in the raster data
landuse <- raster("D:/lust/lus.tif",package="raster")
## Extract attribute information for the categories from the raster layer, and convert into dataframe
LanduseAttributeTable <- ldply(landuse@data@attributes, data.frame)
#head(LanduseAttributeTable)

In [ ]:
## Read in the raster data
Livestock <- raster("D:/lvstdt/lvstd.tif",package="raster")
## Extract attribute information for the categories from the raster layer, and convert into dataframe
LivestockAttributeTable <- ldply(Livestock@data@attributes, data.frame)
#LivestockAttributeTable

In [ ]:
## Read in the raster data
Slope <- raster("D:/slpt/slp.tif",package="raster")
## Extract attribute information for the categories from the raster layer, and convert into dataframe
SlopeAttributeTable <- ldply(Slope@data@attributes, data.frame)
#SlopeAttributeTable

In [ ]:
model.data = merge(model.data, LanduseAttributeTable, by.x = 'LandUse_majority', by.y = 'Value', all.x = TRUE)   
model.data = merge(model.data, LivestockAttributeTable, by.x = 'Livestock_majority', by.y = 'Value', all.x = TRUE)  
model.data = merge(model.data, SlopeAttributeTable, by.x = 'Slope_majority', by.y = 'Value', all.x = TRUE)

In [ ]:
unique(model.data$LandUse_collapsed)

In [ ]:
table(model.data$SLP_DS)

In [ ]:
write.csv(model.data, 'Outputs/PartialDependence/Group4Data_forPartial.csv')

In [ ]:
habitats = c("Forest","Savanna","Shrubland","Grassland","Wetlands","Rocky.areas","Caves.and.Subterranean.Habitats","Desert",
            "Marine.Neritic","Marine.Oceanic","Marine.Deep.Ocean.Floor","Marine.Intertidal","Marine.Coastal.Supratidal", 
            "Artificial.Terrestrial","Artificial.Aquatic","Introduced.Vegetation","Other","Unknown")

predictorsNames = c("Bio12_majority", "Bio12_max", "Bio12_mean", "Bio12_min", "Bio12_range",
                    "Bio12_std", "Bio1_majority", "Bio1_max", "Bio1_mean", "Bio1_min", "Bio1_range", "Bio1_std", "Bio3_majority",
                    "Bio3_max", "Bio3_mean", "Bio3_min", "Bio3_range", "Bio3_std", "Crops_majority",
                    "HuPop_majority", "HuPop_max", "HuPop_mean", "HuPop_min", "HuPop_range",
                    "HuPop_std", "LandUse_majority", "Livestock_majority", "Mosquitoes_majority", "Mosquitoes_max",
                    "Mosquitoes_mean", "Slope_mean", "Slope_min", "Slope_range", "Slope_std",
                    "Ticks_majority", "Ticks_mean", "Ticks_min", "bdiversity_majority", "bdiversity_max",
                    "bdiversity_mean", "bdiversity_min", "bdiversity_range", "bdiversity_std",
                    "centroidY", "maxlat", "mdiversity_majority", "mdiversity_max", "mdiversity_mean",
                    "mdiversity_min", "mdiversity_range", "mdiversity_std", "minlat",
                    "minlong", "Mosquitoes_min_log", "Mosquitoes_range_log", "Mosquitoes_std_log",
                    "Shape_Area_log", "Shape_Length_log", "Slope_majority_log", "Slope_max_log",
                    "Ticks_max_log", "Ticks_range_log", "Ticks_std_log", "centroidX_log", "maxlong_log",
                    "MaxDiet", "ForStrategy", "ActivityTime","Body_Temperature", "IUCN.Status",
                    "Population.trend", "Data.Richness", "BodyMass_ln", "Metabolic_Rate_ln", "Habitat_Breadth_sqrt",
                    "PubmedHits_ln",'HuPop_mean_log',
                    'Class','Family','Order','SLP_DS',
                    'PTEROPODIDAE', 'PHYLLOSTOMIDAE', 'FURIPTERIDAE', 'HIPPOSIDERIDAE',
       'VESPERTILIONIDAE', 'MOLOSSIDAE', 'EMBALLONURIDAE',
       'MEGADERMATIDAE', 'NATALIDAE', 'CRASEONYCTERIDAE', 'MORMOOPIDAE',
       'MYSTACINIDAE', 'MYZOPODIDAE', 'NOCTILIONIDAE', 'NYCTERIDAE',
       'RHINOLOPHIDAE', 'RHINOPOMATIDAE', 'THYROPTERIDAE', 'PubmedHits')


predictorsNames = predictorsNames[!predictorsNames %in% c('Activity_Diurnal', 'Activity_Nocturnal', 'BodyMass_SpecLevel',
                                                          'Diet_Fruit', "Diet_Inv","Diet_Nect","Diet_PlantO", "Diet_Scav",
                                                          "Diet_Seed", "Diet_Vect","Diet_Vend", "Diet_Vfish", "Diet_Vunk",
                                                          "ForStrat_aerial", "ForStrat_aquatic","ForStrat_arboreal", "ForStrat_ground", 
                                                          'Crops_count', "Crops_max","Crops_mean", "Crops_median", "Crops_min","Crops_range",
                                                          "Crops_std","LandUse_count",'LandUse_majority', "LandUse_max","LandUse_mean",
                                                          "LandUse_median","LandUse_min","LandUse_range","LandUse_std","Livestock_count",
                                                          "Livestock_majority","Livestock_max","Livestock_mean","Livestock_median","Livestock_min",
                                                          "Livestock_range","Livestock_std","MetRate_W_log","BodyMass_log",
                                                          'TotalBinary', 'Test', 'Class','Family', 'Order','Data.Richness', 
                                                          'PubmedHits', "BodyMass", "Metabolic.Rate",
                                                          "Distribution.Area", "Mean.Temperature", "Mean.Human.population",
                                                          "HabitatBreadth", "bdiversity_min", "bdiversity_std", "mdiversity_min","mdiversity_std", 
                                                          "DELPHINIDAE", "sp", "BALAENIDAE", "BALAENOPTERIDAE", "ZIPHIIDAE",
                                                          "NEOBALAENIDAE", "CISTUGIDAE", "MONODONTIDAE", "ESCHRICHTIIDAE",
                                                          "INIIDAE", "PHYSETERIDAE", "PHOCOENIDAE", 
                                                          "PLATANISTIDAE","MINIOPTERIDAE", 
                                                          
                                                          "Bio12_range","Bio1_range","Bio3_range", 'HuPop_range',
                                                          "Slope_range", "bdiversity_range", "mdiversity_range",
                                                          "Mosquitoes_range_log","Ticks_range_log","Bio12_majority",
                                                          
                                                          "Bio1_majority","Bio3_majority", 'HuPop_majority',
                                                         "Slope_majority_log", "bdiversity_majority", "mdiversity_majority",
                                                         "Mosquitoes_majority","Ticks_majority",
                                                         
                                                          "Bio1_max","Bio3_max", 'HuPop_max',
                                                         "Slope_max_log", "bdiversity_max", "mdiversity_max",
                                                         "Mosquitoes_max","Ticks_max",
                                                         
                                                          "Bio1_min","Bio3_min", 'HuPop_min',
                                                         "Slope_min_log", "bdiversity_min", "mdiversity_min",
                                                         "Mosquitoes_min","Ticks_min",
                                                         
                                                          "Bio1_std","Bio3_std", 'HuPop_std',
                                                         "Slope_std_log", "bdiversity_std", "mdiversity_std",
                                                         "Mosquitoes_std","Ticks_std",
                                                         
                                                          'Shape_Length_log', "Slope_mean",
                                                         
                                                          "Bio12_max","Bio12_min","Bio12_std",
                                                         "Slope_min","Slope_std", 
                                                         "Mosquitoes_min_log", "Mosquitoes_std_log",
                                                          "Ticks_max_log", "Ticks_std_log", 
                                                         'HuPop_mean')]

predictorsNames = predictorsNames[!predictorsNames %in% habitats]

#predictorsNames = c(predictorsNames, 'LVSTD_DS', 'LUS_DS')
predictorsNames = c(predictorsNames, 'LVSTD_DS', 'LandUse_collapsed')

In [ ]:
outcomeName <- 'Group4Binary'

In [ ]:
predictorsNames

In [ ]:
length(predictorsNames)

## Holdout dataset and Cross Validation

In [ ]:
library(caTools)
model.data$spl=sample.split(model.data$Group4Binary,SplitRatio=0.8)
train=subset(model.data, model.data$spl==TRUE)
test=subset(model.data, model.data$spl==FALSE)
nrow(train)
nrow(test)
table(model.data$Group4Binary)
table(train$Group4Binary)
table(test$Group4Binary)

In [ ]:
set.seed(100)
Group1.total.tc5.lr0005 <- gbm.step(data = train,
                                   gbm.x = predictorsNames,
                                   gbm.y = outcomeName,
                                   family = "bernoulli",
                                   tree.complexity = 5,
                                   learning.rate = 0.001,
                                   bag.fraction = 0.5, 
                                   cv = 10, 
                                   max.trees = 20000)

In [ ]:
print(Group1.total.tc5.lr0005)

In [ ]:
preds2 <- predict.gbm(Group1.total.tc5.lr0005, 
                     test[,predictorsNames],
                     n.trees=Group1.total.tc5.lr0005$gbm.call$best.trees,
                     type="response")

calc.deviance(obs=test$Group4Binary, pred=preds2, calc.mean=TRUE)

In [ ]:
d <- cbind(test$Group4Binary, preds2)
pres <- d[d[,1]==1, 2]
abs <- d[d[,1]==0, 2]
e <- evaluate(p=pres, a=abs)
e

In [ ]:
preds2 <- predict.gbm(Group1.total.tc5.lr0005, 
                     model.data[,predictorsNames],
                     n.trees=Group1.total.tc5.lr0005$gbm.call$best.trees,
                     type="response")

In [ ]:
model.data$PredictionProbability  = preds2
write.csv(model.data, file = "Outputs/Predictions_R_Group4.csv")

In [ ]:
summary(Group1.total.tc5.lr0005)

In [ ]:
ranks = relative.influence(Group1.total.tc5.lr0005, scale. = 'TRUE', sort. = 'TRUE')
a = as.data.frame(ranks, col.names = , optional = FALSE)
a = setDT(a, keep.rownames = TRUE)[]
colnames(a) = c('Variable','influence')
b = a
for (i in b$Variable){
    margianleffects = plot.gbm(Group1.total.tc5.lr0005, i.var = i,n.plots=3, 
                           write.title = T, return.grid  = T,
                           show.contrib = T,y.label="Marginal effect on Host")
    filename = paste("Outputs/PartialDependence/", i,'Group4','.csv', sep="")
    write.csv(margianleffects, file = filename)    
}

In [ ]:
write.csv(a, 'Outputs/PartialDependence/Ranks_group4.csv')